In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import time
from py2neo import Node, Relationship, Graph, Subgraph, NodeMatcher

from req_analysis import Model, Requirement, ContextCluster

In [8]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://ec2-18-224-170-154.us-east-2.compute.amazonaws.com:3001/sparql")

In [9]:
queryString = """prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix xml: <http://www.w3.org/XML/1998/namespace/>
prefix mms-ontology: <https://opencae.jpl.nasa.gov/mms/rdf/ontology/>
prefix mms-graph: <https://opencae.jpl.nasa.gov/mms/rdf/graph/>
prefix mms-property: <https://opencae.jpl.nasa.gov/mms/rdf/property/>
prefix mms-class: <https://opencae.jpl.nasa.gov/mms/rdf/class/>
prefix mms-element: <https://opencae.jpl.nasa.gov/mms/rdf/element/>
prefix mms-artifact: <https://opencae.jpl.nasa.gov/mms/rdf/artifact/>
prefix mms-index: <https://opencae.jpl.nasa.gov/mms/rdf/index/>
prefix xmi: <http://www.omg.org/spec/XMI/20131001#>
prefix uml: <http://www.omg.org/spec/UML/20161101#>
prefix uml-model: <https://www.omg.org/spec/UML/20161101/UML.xmi#>
prefix uml-primitives: <https://www.omg.org/spec/UML/20161101/PrimitiveTypes.xmi#>
prefix uml-class: <https://opencae.jpl.nasa.gov/mms/rdf/uml-class/>
prefix uml-property: <https://opencae.jpl.nasa.gov/mms/rdf/uml-property/>

select * from mms-graph:data.tmt {
    ?subject ?edge ?str .

    values ?edge {
        mms-property:bodyStringFromComment
        mms-property:valueString
    }
    
    filter(isLiteral(?str) && ?str != "")
}
"""

In [10]:
queryElements = """prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix xml: <http://www.w3.org/XML/1998/namespace/>
prefix mms-ontology: <https://opencae.jpl.nasa.gov/mms/rdf/ontology/>
prefix mms-graph: <https://opencae.jpl.nasa.gov/mms/rdf/graph/>
prefix mms-property: <https://opencae.jpl.nasa.gov/mms/rdf/property/>
prefix mms-class: <https://opencae.jpl.nasa.gov/mms/rdf/class/>
prefix mms-element: <https://opencae.jpl.nasa.gov/mms/rdf/element/>
prefix mms-artifact: <https://opencae.jpl.nasa.gov/mms/rdf/artifact/>
prefix mms-index: <https://opencae.jpl.nasa.gov/mms/rdf/index/>
prefix xmi: <http://www.omg.org/spec/XMI/20131001#>
prefix uml: <http://www.omg.org/spec/UML/20161101#>
prefix uml-model: <https://www.omg.org/spec/UML/20161101/UML.xmi#>
prefix uml-primitives: <https://www.omg.org/spec/UML/20161101/PrimitiveTypes.xmi#>
prefix uml-class: <https://opencae.jpl.nasa.gov/mms/rdf/uml-class/>
prefix uml-property: <https://opencae.jpl.nasa.gov/mms/rdf/uml-property/>

select * from mms-graph:data.tmt {
    ?element a/rdfs:subClassOf* uml-class:Class
    {
        ?element rdfs:label ?label 
    } union {
        ?element mms-property:name ?label
    }
    
    filter(isLiteral(?label) && ?label != "")
}
"""

In [11]:
sparql.setQuery(queryElements)

sparql.setReturnFormat(JSON)
el_results = sparql.query().convert()

In [12]:
model_elements = []
for result in el_results["results"]["bindings"]:
#     print(result['element']['value'], result['label']['value'])
    model_elements.append(result['label']['value'])

In [13]:
for result in el_results["results"]["bindings"][0:10]:
    print(result)

{'element': {'type': 'uri', 'value': 'https://opencae.jpl.nasa.gov/mms/rdf/element/_2910_0440c73a-d618-4d1a-814b-3bd81fd9de81'}, 'label': {'type': 'literal', 'value': 'Achievable contrast with coronagraph'}}
{'element': {'type': 'uri', 'value': 'https://opencae.jpl.nasa.gov/mms/rdf/element/_6661c442-ae86-4f4e-bfeb-45c7062378f6__923_6d65203e-dec3-4a7b-92f6-472740aa45dd'}, 'label': {'type': 'literal', 'value': 'Dust'}}
{'element': {'type': 'uri', 'value': 'https://opencae.jpl.nasa.gov/mms/rdf/element/_4_dc0db621-772c-4b5c-80bd-aa50754ffd22'}, 'label': {'type': 'literal', 'value': 'Performance Allocation and System Budgets'}}
{'element': {'type': 'uri', 'value': 'https://opencae.jpl.nasa.gov/mms/rdf/element/_4663af6e-6265-438d-87b8-74e1550518bf__159_44a1bf5a-fc37-474d-ae80-810446dfeeea'}, 'label': {'type': 'literal', 'value': 'Site Construction Camp'}}
{'element': {'type': 'uri', 'value': 'https://opencae.jpl.nasa.gov/mms/rdf/element/_4663af6e-6265-438d-87b8-74e1550518bf__228_c7fb18c9-2ce

In [14]:
sparql.setQuery(queryString)

sparql.setReturnFormat(JSON)
results = sparql.query().convert()



In [15]:
for result in results["results"]["bindings"][0:1100]:
    if str.startswith(result["str"]["value"], "<html>"):
            req_object = Requirement(result["subject"]["value"], result["str"]["value"])
            matches = req_object.match_req_tokens(model_elements, 0.025)
            if len(matches)>0:
                print('Req ID: ', result["subject"]["value"], '\nReq text:' , (result["str"]["value"]))
                print(matches, '\n___________')

Req ID:  https://opencae.jpl.nasa.gov/mms/rdf/element/_4663af6e-6265-438d-87b8-74e1550518bf__1015_e548608e-442a-4e4e-916c-f5c6dba77d99_asi-slot-_17_0_2_3_b4c02e1_1379368975825_138301_38788-slotvalue-0-literalstring 
Req text: <html><body>
<span xmlns="http://www.w3.org/1999/xhtml"><br/>(CCR24) Transponder based system added to improve reliability of detection.<br/>All sky cameras will patrol a large area around the observatory to ensure lasers are shut down before possible illumination of aircraft. Bore sighted camera field of view is immediately around projected laser beam</span></body></html>
[{'token': 'reliability', 'model_element': 'Reliability', 'score': 1.998401444325281e-17}, {'token': 'lasers', 'model_element': 'Laser', 'score': 0.013855509526361283}, {'token': 'laser', 'model_element': 'Laser', 'score': 4.440892098500625e-17}] 
___________
Req ID:  https://opencae.jpl.nasa.gov/mms/rdf/element/_4663af6e-6265-438d-87b8-74e1550518bf__6335_3c300526-76ad-407c-9a75-61cd1296b7eb_asi